In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib widget
plt.ion()
sns.set(rc={'figure.figsize': (11, 4)})
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
# Load data
df = pd.read_csv("../data/imputed/data_imp1.csv", index_col=0)
df = df.dropna()

In [4]:
metadata = pd.read_csv("../data/df_sites.csv")

In [5]:
# Group data by sitename
sites = df['sitename'].unique()
sites_df = [df[df['sitename'] == site] for site in sites]

for i in range(len(sites_df)):
    sites_df[i]['date'] = pd.to_datetime(sites_df[i]['date'], format="%Y-%m-%d")
    sites_df[i] = sites_df[i].set_index("date")

In [8]:
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from tqdm import tqdm

regr =  GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,  max_depth=2, random_state=0, loss='ls')
cv = []
sites = df['sitename'].unique()


for i in tqdm(range(len(sites))):
    train = df[df.sitename != sites[i]]
    test = df[df.sitename == sites[i]]
    X_train = train.drop(columns=["GPP_NT_VUT_REF","sitename","date"]).values
    X_test = test.drop(columns=["GPP_NT_VUT_REF","sitename","date"]).values
    y_train = train['GPP_NT_VUT_REF'].values
    y_test = test['GPP_NT_VUT_REF'].values
    regr.fit(X_train,y_train)
    preds = regr.predict(X_test)
    cv.append((sites[i],mean_squared_error(y_test,preds)))




  0%|          | 0/71 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [347]:
cv_np = np.asarray(cv)[:,1].astype(float)
print(f"LOOCV: {np.mean(cv_np)} +- {np.std(cv_np)}")

LOOCV: 7.514659705557011 +- 3.410238044429465


In [15]:
regr =  GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,  max_depth=2, random_state=0, loss='ls')
train = df[df.sitename != sites[30]]
test = df[df.sitename == sites[30]]
X_train = train.drop(columns=["GPP_NT_VUT_REF","sitename","date"]).values
X_test = test.drop(columns=["GPP_NT_VUT_REF","sitename","date"]).values
y_train = train['GPP_NT_VUT_REF'].values
y_test = test['GPP_NT_VUT_REF'].values
regr.fit(X_train,y_train)


GradientBoostingRegressor(max_depth=2, random_state=0)

In [16]:
%matplotlib widget
preds = regr.predict(X_test)
index = sites_df[30].index
plt.ion()
plt.scatter(index, preds, label="Predictions", s=1)
plt.scatter(index, y_test, label="Ground truth"+" "+sites_df[65]['sitename'].unique(), s=1)
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [304]:
residuals = (preds - y_test) ** 2

%matplotlib widget
plt.ion()
plt.scatter(index, residuals, label="Residuals", s=1)
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [305]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,preds)

10.983337980311918